# Karim Kuserbaev

## Импорт библиотек и настройки

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
# pip install catboost

In [44]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import os

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
import catboost
from catboost import * 
from catboost import datasets

In [45]:
!pip freeze | grep "numpy\|pandas\|lightgbm\|scikit-learn"

geopandas==0.14.3
lightgbm @ file:///tmp/lightgbm/lightgbm-4.2.0-py3-none-manylinux_2_31_x86_64.whl#sha256=26ed21477c12bb26edc4d6d51336cd43d5a8f7daf55ebbe27b0faf50ce96db23
numpy @ file:///home/conda/feedstock_root/build_artifacts/numpy_1707225380409/work/dist/numpy-1.26.4-cp310-cp310-linux_x86_64.whl#sha256=51131fd8fc130cd168aecaf1bc0ea85f92e8ffebf211772ceb16ac2e7f10d7ca
pandas==2.1.4
pandas-datareader==0.10.0
pandas-profiling==3.6.6
pandas-summary==0.2.0
pandasql==0.7.3
scikit-learn==1.2.2
scikit-learn-intelex==2024.2.0
sklearn-pandas==2.2.0


In [46]:
pd.options.display.max_rows = 93
pd.options.display.max_columns = 93

## Загрузка данных

In [47]:
train_df = pd.read_parquet("/kaggle/input/alfa-hack-dataset/restored_train.pqt")
test_df = pd.read_parquet("/kaggle/input/alfahack-test/restored_test.pqt")

## Анализ

Размерность

In [48]:
train_df.shape, test_df.shape

((600000, 93), (290120, 92))

In [49]:
train_df.head()

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.452182,2.736461,2.932560,2.437979,okved_30,segment_1,0.942275,0.536013,0.511490,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.396523,0.893064,0.463405,0.505823,0.873795,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,0.099899,0.226247,0.391439,1.054211,0.654057,0.387856,1.171020,0.648942,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,0.356556,0.426034,1.201541,0.435983,0.864246,1.270517,-0.152107,0.286074,0.891856,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.186820,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.469917,2.756014,2.952725,2.455375,okved_30,segment_1,0.645704,0.536378,0.486425,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.088209,0.969891,-0.063678,0.221993,0.131636,0.893042,0.398889,0.095082,0.873685,0.526503,-0.101797,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.021465,0.224276,0.326923,0.419979,0.652830,0.258824,0.410410,0.633498,0.564854,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,-0.084701,0.307935,0.302226,0.425993,1.135607,0.411205,0.864251,1.237550,-0.142187,0.286081,0.902845,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.125043,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.485936,2.773674,2.970940,2.472771,okved_30,segment_1,0.403604,0.613167,0.480547,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.485924,0.893049,0.560179,0.148894,0.873656,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.029795,0.223291,0.294665,1.251369,0.653444,0.387856,0.552757,0.629637,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,-0.084701

Количество строк без пропусков

In [50]:
train_df.dropna().shape

(173529, 93)

In [52]:
test_df.dropna().shape

(82665, 92)

## Предобработка

Обозначение категориальных признаков

In [53]:
cat_names = [
    "channel_code", "city", "city_type",
    "index_city_code", "ogrn_month", "ogrn_year",
    "okved", "segment", "start_cluster"
]

In [54]:
label_encoders = {}

for col in cat_names:
    label_encoders[col] = LabelEncoder().fit(train_df[col])
    train_df[col] = label_encoders[col].transform(train_df[col])
    test_df[col] = label_encoders[col].transform(test_df[col])

### Создаем выборки для валидации и обучения

Деление на обучающую и тестовую, исправляем дизбаланс классов, если не требуется, пропустить эти блоки до следующего текста

In [55]:
X = train_df.drop(["id", "date"], axis=1) # оставляю end_cluster чтобы получить пропорцию классов, а потом ниже удалю в коде
y = train_df["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [56]:
# Calculate current class proportions
class_proportions = x_train['end_cluster'].value_counts(normalize=True)

# Identify underrepresented classes
underrepresented_classes = class_proportions[class_proportions < 0.08].index

# Create an empty list to store additional rows
additional_rows = []

# Calculate the number of additional rows needed for each underrepresented class
for class_label in underrepresented_classes:
    current_proportion = class_proportions[class_label]
    desired_proportion = 0.08
    current_count = (x_train['end_cluster'] == class_label).sum()
    desired_count = int(desired_proportion * len(train_df))
    additional_count = desired_count - current_count
    
    # Sample additional rows from the original dataset
    additional_samples = x_train[x_train['end_cluster'] == class_label].sample(n=additional_count, replace=True)
    additional_rows.append(additional_samples)

# Concatenate additional rows with the original dataframe
if additional_rows:
    x_train = pd.concat([x_train] + additional_rows, ignore_index=True)

# Verify the class proportions after balancing
print(x_train['end_cluster'].value_counts(normalize=True))


end_cluster
{α}          0.237582
{}           0.091356
{α, η}       0.044737
{α, θ}       0.044737
{α, γ}       0.044737
{α, μ}       0.044737
{α, ε, θ}    0.044737
{α, ε}       0.044737
{α, ε, η}    0.044737
{other}      0.044737
{α, β}       0.044737
{α, δ}       0.044737
{α, ψ}       0.044737
{α, λ}       0.044737
{α, ε, ψ}    0.044737
{λ}          0.044737
{α, π}       0.044737
Name: proportion, dtype: float64


In [57]:
y_train = x_train['end_cluster']
x_train = x_train.drop(['end_cluster'], axis=1)
x_val = x_val.drop(['end_cluster'], axis=1)

In [58]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((1072926, 90), (1072926,), (120000, 90), (120000,))

Деление на тестовую и обучающую выборку без дизбаланса

In [31]:
X = train_df.drop(["id", "date", "end_cluster"], axis=1)
y = train_df["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

## Обучение модели

Кросс-валидация для CatBoost (не нажимать если не разбираешься). Если не разбираешься, идти к следующему блоку текста

In [20]:
from catboost import cv

params = {
    'loss_function': 'MultiClass',
    'iterations': 512,
    'random_seed': 43,
    'learning_rate': 0.1,
    'depth': 12,
    'task_type': 'GPU',
    'devices':'0'
}

cv_data = cv(
    params=params,
    pool=pool1,
    fold_count=5, # Разбивка выборки на 5 кусочков
    shuffle=True, # Перемешаем наши данные
    partition_random_seed=0,
    plot=True, # Никуда без визуализатора
    stratified=True, 
    verbose=30
)

In [ ]:
cv_data

In [ ]:
best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])
print("Best validation Logloss score, stratified: {:.4f}+/-{:.3f} on step {}".format(
best_value, cv_data['test-Logloss-std'][best_iter], best_iter))

## Начало обучения

### CatBoostClassifier

model #14, classes balanced

In [59]:
model = CatBoostClassifier(iterations=2048,
                           depth=5,
                           learning_rate=0.075,
                           random_seed=43,
                           loss_function='MultiClass',
                           task_type="GPU",
                           devices='0',
                           early_stopping_rounds=20
                          )

In [60]:
def train_catboost(model, x_train, y_train, x_val, y_val, cat_names):
    
    model.fit(
    x_train, y_train,
    cat_features=np.array(cat_names),
    eval_set=(x_val, y_val),
    verbose=15 # через сколько итераций выводить стату
    )
    model.save_model('catboost_model_14.json') # сохранение модели
    feature_importance = model.get_feature_importance(prettified=True) # датасет с важностью признаков
    
    return feature_importance

In [61]:
train_catboost(model, x_train, y_train, x_val, y_val, cat_names)

0:	learn: 2.6224392	test: 2.5292438	best: 2.5292438 (0)	total: 99ms	remaining: 3m 22s
15:	learn: 1.8158042	test: 1.5386846	best: 1.5386846 (15)	total: 1.46s	remaining: 3m 4s
30:	learn: 1.6292191	test: 1.3591801	best: 1.3591801 (30)	total: 2.73s	remaining: 2m 57s
45:	learn: 1.5272682	test: 1.2790669	best: 1.2790669 (45)	total: 4.02s	remaining: 2m 55s
60:	learn: 1.4521892	test: 1.2405945	best: 1.2405945 (60)	total: 5.29s	remaining: 2m 52s
75:	learn: 1.4028916	test: 1.2129349	best: 1.2129349 (75)	total: 6.52s	remaining: 2m 49s
90:	learn: 1.3645262	test: 1.1944870	best: 1.1944870 (90)	total: 7.78s	remaining: 2m 47s
105:	learn: 1.3293272	test: 1.1800737	best: 1.1800737 (105)	total: 9.09s	remaining: 2m 46s
120:	learn: 1.3002014	test: 1.1663512	best: 1.1663512 (120)	total: 10.4s	remaining: 2m 44s
135:	learn: 1.2759400	test: 1.1582370	best: 1.1582370 (135)	total: 11.6s	remaining: 2m 43s
150:	learn: 1.2546343	test: 1.1505911	best: 1.1505911 (150)	total: 12.9s	remaining: 2m 41s
165:	learn: 1.237

,Feature Id,Importances
0,start_cluster,11.196854
1,okved,7.981756
2,city,6.196965
3,index_city_code,5.374536
4,channel_code,4.522525
5,ogrn_year,3.547100
6,segment,3.302146
7,balance_amt_min,3.144222
8,ogrn_month,3.006615
9,ogrn_days_end_quarter,2.664715


Модель LGBM, блоки кода не нажимать чтобы не сбросилась model CatBoost. Я еще не тестил эту модель, переходи к метрикам

### LGBM

В качестве базовой модели возьмем LGBM обучим на всех признаках

In [ ]:
model = LGBMClassifier(verbosity=-1, random_state=42, n_jobs=-1)
model.fit(x_train, y_train)

LGBMClassifier(n_jobs=-1, random_state=42, verbosity=-1)

### Метрики

Зададим функцию для взвешенной метрики roc auc

In [62]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [63]:
cluster_weights = pd.read_excel("/kaggle/input/alfa-hack-dataset/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

Проверка работы модели

In [64]:
y_pred_proba = model.predict_proba(x_val)
y_pred_proba.shape

(120000, 17)

In [65]:
weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.941704337444649

## Прогноз на тестовой выборке

In [67]:
sample_submission_df = pd.read_csv("/kaggle/input/sample-submission/sample_submission.csv") # поменять на свой
last_m_test_df = test_df[test_df["date"] == "month_6"]
last_m_test_df = last_m_test_df.drop(["id", "date"], axis=1)

pool2 = Pool(data=last_m_test_df, cat_features=np.array(cat_names))

test_pred_proba = model.predict_proba(pool2) # last_m_test_df
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("catboost_submission_14.csv", index=False) # сохранение сабмишна

In [69]:
sample_submission_df.shape # если (100000, 18), то все правильно. Если нет - надо писать Кариму и спрашивать(

(100000, 18)

In [43]:
sample_submission_df

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.007460,0.020608,0.015734,0.020771,0.004723,0.000223,0.002583,0.000414,0.012902,0.004112,0.013124,0.000420,0.001409,2.550970e-06,0.004943,0.890568,0.000004
1,200001,0.007815,0.585300,0.000751,0.002789,0.000477,0.000131,0.000342,0.000017,0.001474,0.009696,0.001354,0.000246,0.001162,1.098554e-06,0.000460,0.387893,0.000093
2,200002,0.551448,0.007806,0.004249,0.109117,0.011465,0.001495,0.004969,0.022567,0.036500,0.014153,0.019045,0.003322,0.003120,7.012314e-06,0.057451,0.153170,0.000116
3,200003,0.031708,0.622938,0.000527,0.001284,0.000422,0.000458,0.000291,0.000038,0.000947,0.022773,0.001824,0.000066,0.001097,7.712932e-07,0.000309,0.315307,0.000010
4,200004,0.051290,0.116743,0.009292,0.010022,0.002499,0.003166,0.000286,0.000052,0.005017,0.074383,0.003387,0.000154,0.006306,1.135686e-06,0.000230,0.717169,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.012969,0.281477,0.002704,0.005018,0.001651,0.000018,0.000303,0.000006,0.001416,0.000731,0.001229,0.001093,0.000140,4.133161e-05,0.000793,0.690121,0.000289
99996,299996,0.021505,0.032758,0.019639,0.042006,0.007896,0.000122,0.000717,0.000077,0.008555,0.004343,0.008914,0.005407,0.001857,2.349691e-05,0.003758,0.842242,0.000182
99997,299997,0.025145,0.028019,0.025910,0.067886,0.011194,0.000099,0.001813,0.001360,0.010831,0.003711,0.008134,0.001267,0.002604,2.051336e-05,0.011368,0.800634,0.000005
99998,299998,0.049710,0.332012,0.010496,0.031127,0.003145,0.000759,0.000954,0.000549,0.010695,0.013390,0.006556,0.000982,0.000650,5.967625e-05,0.005820,0.531954,0.001142
